![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [125]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

rental_info = pd.read_csv('rental_info.csv')
rental_info.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [126]:
rental_info.dtypes

rental_date          object
return_date          object
amount              float64
release_year        float64
rental_rate         float64
length              float64
replacement_cost    float64
special_features     object
NC-17                 int64
PG                    int64
PG-13                 int64
R                     int64
amount_2            float64
length_2            float64
rental_rate_2       float64
dtype: object

In [127]:
rental_info['rental_length_days'] = (pd.to_datetime(rental_info['return_date']) - pd.to_datetime(rental_info['rental_date'])).dt.days
rental_info['rental_length_days'].head(3)

0    3
1    2
2    7
Name: rental_length_days, dtype: int64

In [128]:
#rental_info['special_features'] == 'Deleted Scenes'
rental_info['deleted_scenes'] = rental_info['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in x else 0)
rental_info['behind_the_scenes'] = rental_info['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in x else 0)
rental_info['deleted_scenes'].value_counts()

0    7973
1    7888
Name: deleted_scenes, dtype: int64

In [129]:
rental_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rental_date         15861 non-null  object 
 1   return_date         15861 non-null  object 
 2   amount              15861 non-null  float64
 3   release_year        15861 non-null  float64
 4   rental_rate         15861 non-null  float64
 5   length              15861 non-null  float64
 6   replacement_cost    15861 non-null  float64
 7   special_features    15861 non-null  object 
 8   NC-17               15861 non-null  int64  
 9   PG                  15861 non-null  int64  
 10  PG-13               15861 non-null  int64  
 11  R                   15861 non-null  int64  
 12  amount_2            15861 non-null  float64
 13  length_2            15861 non-null  float64
 14  rental_rate_2       15861 non-null  float64
 15  rental_length_days  15861 non-null  int64  
 16  dele

In [130]:
rental_info.corr()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
amount,1.000000,0.021726,0.685870,0.018947,-0.026725,0.003968,-0.010591,0.012773,-0.007029,0.956141,0.017864,0.678597,0.551593,-0.010035,-0.014005
release_year,0.021726,1.000000,0.037304,0.031088,0.069991,0.027187,-0.022237,0.027442,-0.052645,0.015941,0.030640,0.025106,0.007044,0.010470,-0.002617
rental_rate,0.685870,0.037304,1.000000,0.055224,-0.064787,0.036280,0.000174,0.022812,-0.033648,0.587627,0.053390,0.982489,-0.001060,-0.046163,-0.000578
length,0.018947,0.031088,0.055224,1.000000,0.026976,-0.030133,-0.049304,0.057023,0.068685,0.015765,0.987667,0.051516,-0.004547,-0.000409,0.006011
replacement_cost,-0.026725,0.069991,-0.064787,0.026976,1.000000,-0.001685,-0.077158,0.044224,0.017768,-0.018281,0.029747,-0.065835,0.015684,0.046730,0.014281
NC-17,0.003968,0.027187,0.036280,-0.030133,-0.001685,1.000000,-0.254017,-0.272206,-0.252767,0.001186,-0.029444,0.038815,0.000783,0.024747,0.030828
PG,-0.010591,-0.022237,0.000174,-0.049304,-0.077158,-0.254017,1.000000,-0.268408,-0.249240,-0.012859,-0.053299,-0.001420,-0.008066,0.057976,-0.022736
PG-13,0.012773,0.027442,0.022812,0.057023,0.044224,-0.272206,-0.268408,1.000000,-0.267087,0.008954,0.062629,0.022525,0.010201,-0.027855,-0.000388
R,-0.007029,-0.052645,-0.033648,0.068685,0.017768,-0.252767,-0.249240,-0.267087,1.000000,-0.004797,0.059310,-0.033067,-0.007961,-0.041887,0.000771
amount_2,0.956141,0.015941,0.587627,0.015765,-0.018281,0.001186,-0.012859,0.008954,-0.004797,1.000000,0.014662,0.596622,0.549412,0.002433,-0.007377


In [131]:
X = rental_info[['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
       'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2',
        'deleted_scenes', 'behind_the_scenes']].values
y = rental_info['rental_length_days'].values

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((12688, 14), (12688,), (3173, 14), (3173,))

In [133]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse

2.032919189224878

In [134]:
best_model = rf_model
best_mse = mse

In [135]:
# Start your coding from below
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# For lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Run OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Read in data
df_rental = pd.read_csv("rental_info.csv")

# Add information on rental duration
df_rental["rental_length"] = pd.to_datetime(df_rental["return_date"]) - pd.to_datetime(df_rental["rental_date"])
df_rental["rental_length_days"] = df_rental["rental_length"].dt.days

### Add dummy variables
# Add dummy for deleted scenes
df_rental["deleted_scenes"] =  np.where(df_rental["special_features"].str.contains("Deleted Scenes"), 1, 0)
# Add dummy for behind the scenes
df_rental["behind_the_scenes"] =  np.where(df_rental["special_features"].str.contains("Behind the Scenes"), 1, 0)

# Choose columns to drop
cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

# Split into feature and target sets
X = df_rental.drop(cols_to_drop, axis=1)
y = df_rental["rental_length_days"]

# Further split into training and test data
X_train,X_test,y_train,y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.2, 
                                                 random_state=9)

# Create the Lasso model
lasso = Lasso(alpha=0.3, random_state=9) 

# Train the model and access the coefficients
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Perform feature selectino by choosing columns with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# Run OLS models on lasso chosen regression
ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)

# Random forest hyperparameter space
param_dist = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}

# Create a random forest regressor
rf = RandomForestRegressor()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions=param_dist, 
                                 cv=5, 
                                 random_state=9)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

# Create a variable for the best hyper param
hyper_params = rand_search.best_params_

# Run the random forest on the chosen hyper parameters
rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"], 
                           max_depth=hyper_params["max_depth"], 
                           random_state=9)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test, rf_pred)

# Random forest gives lowest MSE so:
best_model = rf
best_mse = mse_random_forest